## Install Dependencies

In [ ]:
! pip install imageAI
! pip install Pillow
!pip install opencv-python
!pip install cython pillow>=7.0.0 numpy>=1.18.1 opencv-python>=4.1.2 torch>=1.9.0 --extra-index-url https://download.pytorch.org/whl/cu102 torchvision>=0.10.0 --extra-index-url https://download.pytorch.org/whl/cu102 pytest==7.1.3 tqdm==4.64.1 scipy>=1.7.3 matplotlib>=3.4.3 mock==4.0.3
!pip install cython pillow>=7.0.0 numpy>=1.18.1 opencv-python>=4.1.2 torch>=1.9.0 --extra-index-url https://download.pytorch.org/whl/cpu torchvision>=0.10.0 --extra-index-url https://download.pytorch.org/whl/cpu pytest==7.1.3 tqdm==4.64.1 scipy>=1.7.3 matplotlib>=3.4.3 mock==4.0.3

In [ ]:
import os
import cv2
import torch
import numpy as np
import joblib
import shutil
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from imageai.Detection import VideoObjectDetection

## Object Detection using Yolo pretrained data

In [ ]:
vid_obj_detect = VideoObjectDetection()
vid_obj_detect.setModelTypeAsYOLOv3()

In [ ]:
vid_obj_detect.setModelPath("yolov3.pt")
vid_obj_detect.loadModel()

In [ ]:
detected_vid_obj = vid_obj_detect.detectObjectsFromVideo(
    input_file_path =  "sample.mp4",
    output_file_path = "output_video",
    frames_per_second=15,
    log_progress=True,
    return_detected_frame = True
)

## Detect frames with person using Yolo

### For each frames, object detection with Yolo, retrieve frames with person detected.

#### Two functions defined : 1 . Retrieve frame number where person are detected. 2. Save detected frames into output folder.

In [ ]:
def forFrame(frame_number, output_array, output_count):
    if 'person' in output_count:
        frame_with_person.append(frame_number)

def save_frames(video_path, frame_numbers, output_directory):
    # Open the video file
    video = cv2.VideoCapture(video_path)
    
    # Iterate over the frame numbers
    for frame_number in frame_numbers:
        # Set the current frame number
        video.set(cv2.CAP_PROP_POS_FRAMES, frame_number)
        
        # Read the frame
        ret, frame = video.read()
        
        # If the frame was read successfully, save it as an image
        if ret:
            output_path = f"{output_directory}/frame_{frame_number}.jpg"
            cv2.imwrite(output_path, frame)
            print(f"Frame {frame_number} saved as {output_path}")
        else:
            print(f"Error reading frame {frame_number}")
    
    # Release the video file
    video.release()



In [ ]:
execution_path = os.getcwd()
frame_with_person = []

## loading Yolo models
video_detector = VideoObjectDetection()
video_detector.setModelTypeAsYOLOv3()
video_detector.setModelPath(os.path.join(execution_path, "yolov3.pt"))
video_detector.loadModel()

## Detecting person in each frame, save detected frames in file output
video_detector.detectObjectsFromVideo(input_file_path=os.path.join(execution_path, "sample.mp4"), output_file_path=os.path.join(execution_path, "video_frame_analysis") ,  frames_per_second=20, per_frame_function=forFrame,  minimum_percentage_probability=30)
video_path = "sample.mp4"
frame_numbers = frame_with_person  # Frame numbers to retrieve
output_directory = "output"

save_frames(video_path, frame_numbers, output_directory)

## ML models for staff detection for all the frames that detected with person previously

### Data Preprocess

10 images for staff exists and 10 images for staff does not exists are selected from the frames retrieved previously. 
Both save in folder **train_yes** and **train_no** respectively. Selected images are removed from output folder.

In [ ]:
# Step 1: Data preparation and preprocessing
# Path to the directories containing the staff and non-staff images
staff_images_dir = 'train_yes'
non_staff_images_dir = 'train_no'

# Load staff images
staff_images = []
for filename in os.listdir(staff_images_dir):
    image = Image.open(os.path.join(staff_images_dir, filename))
    image = image.resize((64, 64))  # Resize the images to a consistent size
    staff_images.append(np.array(image))

# Load non-staff images
non_staff_images = []
for filename in os.listdir(non_staff_images_dir):
    image = Image.open(os.path.join(non_staff_images_dir, filename))
    image = image.resize((64, 64))  # Resize the images to a consistent size
    non_staff_images.append(np.array(image))

# Combine the staff and non-staff images into a single dataset
X = np.concatenate((staff_images, non_staff_images), axis=0)
y = np.concatenate((np.ones(len(staff_images)), np.zeros(len(non_staff_images))), axis=0)

### Train test split, Training and testing data using SVM

In [ ]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the image pixel values to improve model performance
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.reshape(len(X_train), -1))
X_test_scaled = scaler.transform(X_test.reshape(len(X_test), -1))

# Step 2: Model training
# Create a support vector machine (SVM) classifier
svm = SVC(kernel='rbf', C=1.0, random_state=42)

# Train the SVM classifier
svm.fit(X_train_scaled, y_train)

# Step 3: Model evaluation
# Predict the labels for the test set
y_pred = svm.predict(X_test_scaled)

# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


### Predict data not seen by the model (data stored in test folder) for inference, Model saved as pkl file

In [ ]:
# Step 4: Inference
# Load and preprocess a new image for prediction
new_image = Image.open('test/frame_380.jpg')
new_image = new_image.resize((64, 64))  # Resize the image to match the training data
new_image_scaled = scaler.transform(np.array(new_image).reshape(1, -1))

# Make a prediction on the new image
prediction = svm.predict(new_image_scaled)
if prediction[0] == 1:
    print("The image contains a staff.")
else:
    print("The image does not contain a staff.")
    
# Save the trained model
joblib.dump(svm, 'staff_model.pkl')

### For remaining unpredicted images in output folder, predict whether the staff exists

#### After detection, result are separted into two folders, one with staff and one without, frames with staff are displayed.

In [ ]:
images_dir = 'output'
frame_with_staff_tag = []

## load saved model
svm = SVC(kernel='rbf', C=1.0, random_state=42)
svm = joblib.load('staff_model.pkl')

## result folders
staff_folder = 'test_staff_result_frame'
non_staff_folder = 'test_non_staff_result_frame'

# Preprocess images and make predictions
for filename in os.listdir(images_dir):
    image_path = os.path.join(images_dir, filename)
    image = Image.open(image_path)
    image = image.resize((64, 64))  # Resize the image to match the training data

    # Preprocess the image for prediction
    image_scaled = scaler.transform(np.array(image).reshape(1, -1))

    # Make a prediction on the image
    prediction = svm.predict(image_scaled)

    if prediction[0] == 1:
        frame_with_staff_tag.append(filename)
        shutil.copy(image_path, os.path.join(staff_folder, filename))
    else:
        shutil.copy(image_path, os.path.join(non_staff_folder, filename))
        
print("Frames with staff : ")
print(frame_with_staff_tag)